La empresa desea saber si tiene los recursos necesarios para soportar la demanda de los usuarios e
incrementar el numero de ticktes por día. En base a ello, diseñe y desarrolle un modelo y/o script que
permita realizar la simulación del proceso de pasar cuenca-aire con las siguientes especificaciones:\

* La empresa tiene 3 carriles de revisión vehicular

* Un solo carril es habilitado para carros de gran tonelaje, busetas, bus, camiones, solo el 15 % de vehiculos que llegan son de este tipo.
* Cada 3 minutos llega un nuevo vehiculo a revision.
* Se sigue el siguiente proceso:

1. Llega el vehiculo al centro, se estaciona (3-5 minutos), saca un turno (1-3 minutos) y pagan la matricula (5 - 10 minutos).
2. Si esta vacio pasa directo a la revisión que tiene una media de 20 minutos con una desviacion estandar de 10 minutos.
3. Solo el 60 % de vehiculos pasa sin ninguna notificación, el 40% salen realizan el arreglo (2 dias) y nuevamente realizan el proceso.
4. Despues de pasar la revisión, se les pega el sello (1 – 3 minutos) .
5. Finalmente, salen (5 – 10 minutos).

Generar un modelo de simulación por 30 dias y generar las siguientes graficas de resultados:
* Numero de vehiculos atendidos por dia, y por carril
* Porcentaje total de vehiculos aprobados y no aprobados en cuenca aire por dia.
* Tiempos del proceso de matriculación, por dia.

In [2]:
import simpy
import numpy.random as random
from scipy.stats import uniform
from random import expovariate
from termcolor import colored
from scipy import stats

In [21]:
# Tiempo de simulación
DIAS = 30
TIEMPO_SIMULACION = DIAS * 24 * 60 

# Creamos una clase que permita modelar mi escenario
class EMOV(object):
    def __init__(self, environment):
        # Guardamos como variable el entorno de ejecucion
        self.env=environment
        # Creamos el recurso que representa al carril para vehiculos pesados y livianos
        self.carril_unico = simpy.Resource(environment, 1)
        self.carriles = simpy.Resource(environment, 2)
        
    def usar_cajero(self, cliente):
        print(f'El cajero entra en uso por el cliente {cliente} a las {env.now}')
        yield self.env.timeout(random.expovariate(3))
        print(f'El cliente {cliente} termina de usar el cajero a las {env.now}')
        
    def pasar_cuenca_aire(self, tipo_vehiculo, placa):
        print(colored(f'Vehiculo {tipo_vehiculo} con placa {placa} empieza el proceso', 'yellow', attrs=['bold']))
        print(colored(f'Vehiculo {tipo_vehiculo} con placa {placa} se estaciona', 'yellow', attrs=['bold']))
        yield self.env.timeout(self.estacionar())
        print(colored(f'Vehiculo {tipo_vehiculo} con placa {placa} saca turno', 'yellow', attrs=['bold']))
        yield self.env.timeout(self.sacar_turno())
        print(colored(f'Vehiculo {tipo_vehiculo} con placa {placa} paga la matricual', 'yellow', attrs=['bold']))
        yield self.env.timeout(self.pagar_matricula())
        print(colored(f'Vehiculo {tipo_vehiculo} con placa {placa} pasa a revisión', 'yellow', attrs=['bold']))
        yield self.env.timeout(self.pasar_revision())
        
        if self.probabilidad_pasar_sin_notificacion():
            print(colored(f'Se pega el sello al vehiculo {tipo_vehiculo} con placa {placa}', 'yellow', attrs=['bold']))
            yield self.env.timeout(self.pegar_sello())
            print(colored(f'Vehiculo {tipo_vehiculo} con placa {placa} sale tras completar la revisión', 'red', attrs=['bold']))
            yield self.env.timeout(self.salir())
        else:
            print(colored(f'Sale a realizar un arrelgo durante 2 días el vehiculo {tipo_vehiculo} con placa {placa} ', 'yellow', attrs=['bold']))
            #yield self.env.timeout(2880)
            #self.pasar_cuenca_aire(tipo_vehiculo, placa)
    # Paso 1
    def estacionar(self):
        return random.randint(3, 5)
    def sacar_turno(self):
        return random.randint(1, 3)
    def pagar_matricula(self):
        return random.randint(5, 10)
    def pasar_revision(self):
        mu, sigma = 20, 10 # media y desvio estandar
        return abs(random.normal(mu, sigma, 1)[0])
    def pegar_sello(self):
        return random.randint(1, 3)
    def salir(self):
        return random.randint(5, 10)
    def probabilidad_pasar_sin_notificacion(sefl):
        # 1 : 60 %
        # 0: 40 %
        return stats.bernoulli(0.60).rvs(1)[0]

def obtener_placa():
    letras = ["A","B","C","D","E","F","G","H","I","J","K","L","M",
              "N","O","P","Q","R","S","T","U","V","W","X","Y","Z"]
    numeros = ["1","2","3","4","5","6","7","8","9","0"]
    
    a = random.choice(letras)
    b = random.choice(letras)
    c = random.choice(letras)
    d = random.choice(numeros)
    e = random.choice(numeros)
    f = random.choice(numeros)
    
    return a+b+c+d+e+f

def probabilidad_tipo_vehiculo():
    # 1: 15% vehiculo pesado
    # 0: 85% vehiculo liviano
    return stats.bernoulli(0.15).rvs(1)[0]

# Generamos el proceso de cola
def llegada_automovil(env, tipo_vehiculo, placa, emov):
    print(colored(f'LLega vehiculo {tipo_vehiculo} con placa {placa}', 'green', attrs=['bold']))
    
    if tipo_vehiculo == 'LIVIANO':
        with emov.carriles.request() as carril:
        # Ocupamos el carril
            #print(carril)
            yield carril
            yield env.process(emov.pasar_cuenca_aire(tipo_vehiculo, placa))
            #Libera el recurso
            emov.carriles.release(carril)
    else:
        with emov.carril_unico.request() as carril:
            # Ocupamos el carril unico
            #print(carril)
            yield carril
            yield env.process(emov.pasar_cuenca_aire(tipo_vehiculo, placa))
            #Libera el recurso
            emov.carril_unico.release(carril)
        
        
def ejecutar_simulacion(env):
    
    emov=EMOV(env)
    
    while True:
        # Generamos un procesos de llegada cada 3 minutos
        yield env.timeout(3)
        # Generamos un tipo de vehiculo Pesado o Liviano y se Atiende al automovil  
        if probabilidad_tipo_vehiculo():
            env.process(llegada_automovil(env,'PESADO', obtener_placa(), emov))
        else:
            env.process(llegada_automovil(env,'LIVIANO', obtener_placa(), emov))
        

            
print(colored(f'Simulación Iniciada! \n', 'green', attrs=['bold']))
# Creamos el entorno de simulacion
env=simpy.Environment()
env.process(ejecutar_simulacion(env))
# Ejecutamos el proceso
env.run(until=TIEMPO_SIMULACION)

Simulación Iniciada! 

LLega vehiculo LIVIANO con placa YSQ191
Vehiculo LIVIANO con placa YSQ191 empieza el proceso
Vehiculo LIVIANO con placa YSQ191 se estaciona
LLega vehiculo PESADO con placa TWE631
Vehiculo PESADO con placa TWE631 empieza el proceso
Vehiculo PESADO con placa TWE631 se estaciona
Vehiculo LIVIANO con placa YSQ191 saca turno
Vehiculo LIVIANO con placa YSQ191 paga la matricual
LLega vehiculo LIVIANO con placa IEW557
Vehiculo LIVIANO con placa IEW557 empieza el proceso
Vehiculo LIVIANO con placa IEW557 se estaciona
Vehiculo PESADO con placa TWE631 saca turno
Vehiculo PESADO con placa TWE631 paga la matricual
LLega vehiculo LIVIANO con placa SKU197
Vehiculo LIVIANO con placa IEW557 saca turno
Vehiculo LIVIANO con placa IEW557 paga la matricual
LLega vehiculo LIVIANO con placa LVI832
Vehiculo PESADO con placa TWE631 pasa a revisión
Vehiculo LIVIANO con placa YSQ191 pasa a revisión
LLega vehiculo LIVIANO con placa AZF344
LLega vehiculo PESADO con placa LIK611
Vehiculo LIVI

LLega vehiculo LIVIANO con placa HKI726
Vehiculo LIVIANO con placa QVM761 empieza el proceso
Vehiculo LIVIANO con placa QVM761 se estaciona
Vehiculo PESADO con placa KDH732 pasa a revisión
Vehiculo LIVIANO con placa POG527 saca turno
LLega vehiculo LIVIANO con placa WXR785
Vehiculo LIVIANO con placa POG527 paga la matricual
Vehiculo LIVIANO con placa QVM761 saca turno
LLega vehiculo LIVIANO con placa WTZ318
Vehiculo LIVIANO con placa QVM761 paga la matricual
LLega vehiculo LIVIANO con placa JIP321
LLega vehiculo LIVIANO con placa JKS684
Vehiculo LIVIANO con placa POG527 pasa a revisión
Vehiculo LIVIANO con placa QVM761 pasa a revisión
LLega vehiculo LIVIANO con placa ELF253
Sale a realizar un arrelgo durante 2 días el vehiculo LIVIANO con placa QVM761 
Vehiculo LIVIANO con placa ALU185 empieza el proceso
Vehiculo LIVIANO con placa ALU185 se estaciona
LLega vehiculo LIVIANO con placa CSO477
Se pega el sello al vehiculo PESADO con placa KDH732
Vehiculo LIVIANO con placa ALU185 saca turno

Se pega el sello al vehiculo LIVIANO con placa MEB320
LLega vehiculo LIVIANO con placa VCO056
Vehiculo LIVIANO con placa MEB320 sale tras completar la revisión
Vehiculo LIVIANO con placa FGR083 pasa a revisión
Se pega el sello al vehiculo PESADO con placa VLZ746
LLega vehiculo LIVIANO con placa GFY743
Vehiculo PESADO con placa VLZ746 sale tras completar la revisión
LLega vehiculo LIVIANO con placa EFW288
Vehiculo LIVIANO con placa SLI080 empieza el proceso
Vehiculo LIVIANO con placa SLI080 se estaciona
LLega vehiculo LIVIANO con placa WVM637
Vehiculo LIVIANO con placa SLI080 saca turno
Vehiculo PESADO con placa ZLQ674 empieza el proceso
Vehiculo PESADO con placa ZLQ674 se estaciona
Vehiculo LIVIANO con placa SLI080 paga la matricual
LLega vehiculo LIVIANO con placa OXK588
LLega vehiculo LIVIANO con placa VYE345
Vehiculo PESADO con placa ZLQ674 saca turno
Vehiculo PESADO con placa ZLQ674 paga la matricual
Vehiculo LIVIANO con placa SLI080 pasa a revisión
LLega vehiculo LIVIANO con placa

LLega vehiculo LIVIANO con placa QGC697
Vehiculo PESADO con placa XMC226 sale tras completar la revisión
Vehiculo LIVIANO con placa DKD821 empieza el proceso
Vehiculo LIVIANO con placa DKD821 se estaciona
Sale a realizar un arrelgo durante 2 días el vehiculo LIVIANO con placa LYW168 
Vehiculo LIVIANO con placa JWO035 empieza el proceso
Vehiculo LIVIANO con placa JWO035 se estaciona
LLega vehiculo LIVIANO con placa YBP674
Vehiculo LIVIANO con placa DKD821 saca turno
Vehiculo LIVIANO con placa JWO035 saca turno
Vehiculo LIVIANO con placa DKD821 paga la matricual
LLega vehiculo LIVIANO con placa HEW355
Vehiculo LIVIANO con placa JWO035 paga la matricual
LLega vehiculo LIVIANO con placa MDW231
Vehiculo PESADO con placa JHI650 empieza el proceso
Vehiculo PESADO con placa JHI650 se estaciona
Vehiculo LIVIANO con placa DKD821 pasa a revisión
LLega vehiculo LIVIANO con placa NKB982
Vehiculo PESADO con placa JHI650 saca turno
Vehiculo PESADO con placa JHI650 paga la matricual
LLega vehiculo LIV

LLega vehiculo LIVIANO con placa QER795
Vehiculo PESADO con placa QRK537 pasa a revisión
LLega vehiculo PESADO con placa ANE273
Vehiculo LIVIANO con placa VVB383 empieza el proceso
Vehiculo LIVIANO con placa VVB383 se estaciona
LLega vehiculo LIVIANO con placa ASU178
Vehiculo LIVIANO con placa VVB383 saca turno
Se pega el sello al vehiculo PESADO con placa QRK537
LLega vehiculo LIVIANO con placa YNB709
Vehiculo LIVIANO con placa VVB383 paga la matricual
Vehiculo PESADO con placa QRK537 sale tras completar la revisión
LLega vehiculo LIVIANO con placa CCO462
LLega vehiculo LIVIANO con placa RRQ628
Vehiculo PESADO con placa WRH517 empieza el proceso
Vehiculo PESADO con placa WRH517 se estaciona
Se pega el sello al vehiculo LIVIANO con placa TFZ331
LLega vehiculo LIVIANO con placa CTB920
Vehiculo LIVIANO con placa VVB383 pasa a revisión
Vehiculo LIVIANO con placa TFZ331 sale tras completar la revisión
Vehiculo PESADO con placa WRH517 saca turno
Vehiculo PESADO con placa WRH517 paga la matr

...

LLega vehiculo LIVIANO con placa YSG189
LLega vehiculo LIVIANO con placa ZYR692
Sale a realizar un arrelgo durante 2 días el vehiculo PESADO con placa LSV137 
Vehiculo PESADO con placa AOR102 empieza el proceso
Vehiculo PESADO con placa AOR102 se estaciona
LLega vehiculo LIVIANO con placa PLX528
Vehiculo PESADO con placa AOR102 saca turno
Sale a realizar un arrelgo durante 2 días el vehiculo LIVIANO con placa IKX090 
Vehiculo LIVIANO con placa DVM921 empieza el proceso
Vehiculo LIVIANO con placa DVM921 se estaciona
Vehiculo PESADO con placa AOR102 paga la matricual
LLega vehiculo LIVIANO con placa VMC275
Vehiculo LIVIANO con placa DVM921 saca turno
LLega vehiculo LIVIANO con placa JAZ080
Vehiculo LIVIANO con placa DVM921 paga la matricual
Vehiculo PESADO con placa AOR102 pasa a revisión
LLega vehiculo LIVIANO con placa EFE311
Vehiculo LIVIANO con placa DVM921 pasa a revisión
LLega vehiculo LIVIANO con placa VKG627
LLega vehiculo LIVIANO con placa OYJ350
LLega vehiculo LIVIANO con placa

Sale a realizar un arrelgo durante 2 días el vehiculo PESADO con placa FXB650 
Vehiculo PESADO con placa RWV042 empieza el proceso
Vehiculo PESADO con placa RWV042 se estaciona
LLega vehiculo LIVIANO con placa UDW714
Vehiculo PESADO con placa RWV042 saca turno
LLega vehiculo LIVIANO con placa JMA626
Vehiculo PESADO con placa RWV042 paga la matricual
LLega vehiculo LIVIANO con placa IAE971
LLega vehiculo PESADO con placa UZL430
Sale a realizar un arrelgo durante 2 días el vehiculo LIVIANO con placa SEM758 
Vehiculo LIVIANO con placa PHJ723 empieza el proceso
Vehiculo LIVIANO con placa PHJ723 se estaciona
Vehiculo PESADO con placa RWV042 pasa a revisión
LLega vehiculo LIVIANO con placa MXX098
Vehiculo LIVIANO con placa PHJ723 saca turno
Vehiculo LIVIANO con placa PHJ723 paga la matricual
LLega vehiculo LIVIANO con placa QFM289
LLega vehiculo LIVIANO con placa CTV597
Se pega el sello al vehiculo LIVIANO con placa MYD375
Vehiculo LIVIANO con placa PHJ723 pasa a revisión
Vehiculo LIVIANO co

LLega vehiculo LIVIANO con placa GWN119
Vehiculo PESADO con placa TOW587 saca turno
Vehiculo LIVIANO con placa UXZ307 pasa a revisión
LLega vehiculo LIVIANO con placa IRA943
Vehiculo PESADO con placa TOW587 paga la matricual
LLega vehiculo PESADO con placa QAI707
LLega vehiculo LIVIANO con placa EAY650
Vehiculo PESADO con placa TOW587 pasa a revisión
LLega vehiculo LIVIANO con placa PPV494
LLega vehiculo LIVIANO con placa JOB939
Se pega el sello al vehiculo LIVIANO con placa JZE661
Vehiculo LIVIANO con placa JZE661 sale tras completar la revisión
LLega vehiculo LIVIANO con placa GSR257
Sale a realizar un arrelgo durante 2 días el vehiculo LIVIANO con placa UXZ307 
Vehiculo LIVIANO con placa VKA999 empieza el proceso
Vehiculo LIVIANO con placa VKA999 se estaciona
LLega vehiculo LIVIANO con placa SLX831
LLega vehiculo PESADO con placa LSK093
Vehiculo LIVIANO con placa EXG884 empieza el proceso
Vehiculo LIVIANO con placa EXG884 se estaciona
Vehiculo LIVIANO con placa VKA999 saca turno
Veh

LLega vehiculo LIVIANO con placa XCG120
Vehiculo PESADO con placa JCR939 saca turno
LLega vehiculo LIVIANO con placa IDC731
Vehiculo PESADO con placa JCR939 paga la matricual
LLega vehiculo LIVIANO con placa QQE710
LLega vehiculo LIVIANO con placa ZYH212
Vehiculo PESADO con placa JCR939 pasa a revisión
LLega vehiculo LIVIANO con placa FOA357
LLega vehiculo LIVIANO con placa WAF361
LLega vehiculo LIVIANO con placa WWY301
Se pega el sello al vehiculo LIVIANO con placa MOG300
LLega vehiculo LIVIANO con placa NMC201
Vehiculo LIVIANO con placa MOG300 sale tras completar la revisión
LLega vehiculo PESADO con placa DKI595
LLega vehiculo LIVIANO con placa VWN950
Sale a realizar un arrelgo durante 2 días el vehiculo PESADO con placa JCR939 
Vehiculo PESADO con placa OPN467 empieza el proceso
Vehiculo PESADO con placa OPN467 se estaciona
LLega vehiculo PESADO con placa GZH961
Vehiculo LIVIANO con placa MKN875 empieza el proceso
Vehiculo LIVIANO con placa MKN875 se estaciona
Sale a realizar un ar

LLega vehiculo LIVIANO con placa ZZG784
Vehiculo PESADO con placa JFY696 empieza el proceso
Vehiculo PESADO con placa JFY696 se estaciona
LLega vehiculo LIVIANO con placa YBJ248
Sale a realizar un arrelgo durante 2 días el vehiculo LIVIANO con placa RVN679 
Vehiculo LIVIANO con placa MYN368 empieza el proceso
Vehiculo LIVIANO con placa MYN368 se estaciona
Vehiculo PESADO con placa JFY696 saca turno
LLega vehiculo LIVIANO con placa CDB268
Vehiculo PESADO con placa JFY696 paga la matricual
Vehiculo LIVIANO con placa MYN368 saca turno
LLega vehiculo LIVIANO con placa RDK629
Vehiculo LIVIANO con placa MYN368 paga la matricual
LLega vehiculo LIVIANO con placa LPW895
Vehiculo PESADO con placa JFY696 pasa a revisión
LLega vehiculo LIVIANO con placa DFZ540
Vehiculo LIVIANO con placa MYN368 pasa a revisión
LLega vehiculo LIVIANO con placa IWD936
Sale a realizar un arrelgo durante 2 días el vehiculo LIVIANO con placa BMX048 
Vehiculo LIVIANO con placa QBZ393 empieza el proceso
Vehiculo LIVIANO c